# Creating Responses

In [1]:
%cd H:\GitHub\pytracer\

H:\GitHub\pytracer


In [2]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
import numpy as np
from scripts.assemblies import shielded_assembly
from scripts.utils import nice_double_plot
import pytracer.geometry as geo
import pytracer.fission as fission

In [ ]:
assembly_solids = shielded_assembly()
assembly_flat = geo.flatten(assembly_solids)

nu_dist = np.array([0.0481677, 0.2485215, 0.4253044, 0.2284094, 0.0423438, 0.0072533], dtype=np.double)
nu_dist /= np.sum(nu_dist)

In [ ]:
plt.figure()
geo.draw(assembly_solids)

radians = np.linspace(0, np.pi, 100)
arc_radians = np.linspace(-np.pi / 8, np.pi / 8, 100)
source, detector_points, extent = geo.fan_beam_paths(60, arc_radians, radians, extent=True)
source = source[0, :, :]

grid = geo.Grid(width=25, height=15, num_x=25, num_y=15)
grid.draw()

cell_i = 520 / 4 + 5
grid_points = grid.cell(cell_i)
plt.fill(grid_points[:, 0], grid_points[:, 1], color='blue', zorder=12)
plt.xlabel('X (cm)', size=18)
plt.ylabel('Y (cm)', size=18)
ax = plt.gca()
ax.set_aspect('equal')
# plt.scatter(source[0, 0], source[0, 1])

In [ ]:
response_single = fission.grid_response(source, detector_points, detector_points, grid, assembly_flat, 1, nu_dist)
np.save(r'scripts\data\fission_response_single', response_single)
response_single = np.load(r'scripts\data\fission_response_single.npy')

In [ ]:
response_double = fission.grid_response(source, detector_points, detector_points, grid, assembly_flat, 2, nu_dist)
np.save(r'scripts\data\fission_response_double', response_double)
response_double = np.load(r'scripts\data\fission_response_double.npy')

In [ ]:
single_probs = fission.scan(source, detector_points, detector_points, assembly_flat, 1, nu_dist)
np.save(r'scripts\data\single_probs', single_probs)
single_probs = np.load(r'scripts\data\single_probs.npy')

In [ ]:
double_probs = fission.scan(source, detector_points, detector_points, assembly_flat, 2, nu_dist)
np.save(r'scripts\data\double_probs', double_probs)
double_probs = np.load(r'scripts\data\double_probs.npy')

In [ ]:
nice_double_plot(response_single[cell_i].T, response_double[cell_i].T, extent, 'Single Fission Response',
                 'Double Fission Response', 'Detector Orientation Angle', 'Source Neutron Direction Angle')
plt.show()

In [ ]:
nice_double_plot(single_probs.T, double_probs.T, extent, 'Single Neutron Probability',
                 'Double Neutron Probability', 'Detector Orientation Angle', 'Source Neutron Direction Angle')
plt.show()